# Trabajo Práctico Machine Learning: Estadísticas de jugadores NBA

La National Basketball Association, más conocida simplemente por sus siglas NBA, es una liga de baloncesto profesional que se disputa en Estados Unidos desde 1949. La misma esta rankeada como la mejor liga en el mundo de este deporte. Actualmente, la misma posee 30 equipos que compiten entre ellos. 

![Chicago Bulls vs Utah Jazz 1998](https://i.pinimg.com/564x/75/91/60/75916076b50a451a4c09eb58d3beff11.jpg)

## 1. Listado de variables y Selección

In [309]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Image
from IPython.core.display import HTML 

In [310]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras
import h5py
import PIL
import seaborn
import plotly
import sklearn_pandas

In [311]:
#path = 'C:\\Users\\Bagre\\Desktop\\Datos_carrera\\Quinto\\ML\\datasets\\Seasons_Stats.csv' FRANCO
path = 'C:\\Users\\Agu\\Desktop\\University\\Quinto\\1erCuatrimestre\\ML\\Trabajo Practico Nro 1\\archive\\Seasons_Stats.csv'
data = pd.read_csv(path)

In [312]:
columns = ['GS', 'TOV', 'BLK', 'STL', 'AST', 'TRB', 'DRB', 'DRB%', 'PF', 'ORB', 'ORB%', 'AST', 'STL', 'blank2', 'DBPM', 'DWS', 'blanl', 'WS', 'WS/48', 'STL%', 'AST%', 'TRB%', 'G', 'Tm', 'BLK%', 'TOV%', '3PA', '3P%', '2P', '2PA', '2P%', '3PAr', 'FTr', 'FT', 'FTA', 'FT%', '3P']
data = data.drop(columns, axis=1)

In [313]:
data = data.drop(data[data.Year < 1980].index, axis=0)
data = data.dropna()

In [314]:
df = pd.DataFrame(data)

In [315]:
df = df.sort_values(['Player', 'Year'], ascending=[True, True])

In [316]:
df.insert(18, 'morePoints', 'Yes', allow_duplicates = True)

In [317]:
df = df.drop_duplicates(subset=['Player','Year'])

In [318]:
df = df.rename(columns = {'Unnamed: 0' : 'id'})

In [319]:
def funcionSetMorePoints(id):
    auxiliar = df.loc[id]
    temporadaAnterior = df.loc[(df.Player == auxiliar.Player) & (df.Year == auxiliar.Year  - 1)]      
    if (temporadaAnterior.empty):
        df.loc[df.id == id,'morePoints'] = 'Yes'
    else:
        if (auxiliar.PTS > temporadaAnterior.PTS.item()):
            df.loc[df.id == id,'morePoints'] = 'Yes'
        else:
            df.loc[df.id == id,'morePoints'] = 'No'
    #if ((not temporadaAnterior.empty) & (auxiliar.PTS <= temporadaAnterior.PTS.item())):
     #   df.loc[df.id == id,'morePoints'] = 'No'
    return id

In [321]:
df.id = df.id.apply(funcionSetMorePoints)

8035             A.C. Green
8420             A.C. Green
8807             A.C. Green
9242             A.C. Green
9688             A.C. Green
                ...        
18442    Zydrunas Ilgauskas
19003    Zydrunas Ilgauskas
19600    Zydrunas Ilgauskas
20187    Zydrunas Ilgauskas
20802    Zydrunas Ilgauskas
Name: Player, Length: 15476, dtype: object

In [339]:
# Con esta linea de codigo pueden buscar cualquier jugador que quieran y revisar que la
jugador = df[df['Player'] == 'Michael Jordan*']
display(jugador)

,id,Year,Player,Pos,Age,MP,PER,TS%,USG%,OWS,OBPM,BPM,VORP,FG,FGA,FG%,eFG%,PTS,morePoints
7716,7716,1985.0,Michael Jordan*,SG,21.0,3144.0,25.8,0.592,29.8,10.3,6.8,8.2,8.1,837.0,1625.0,0.515,0.518,2313.0,Yes
8094,8094,1986.0,Michael Jordan*,SG,22.0,451.0,27.5,0.533,38.6,1.0,5.1,4.7,0.8,150.0,328.0,0.457,0.462,408.0,No
8469,8469,1987.0,Michael Jordan*,SG,23.0,3281.0,29.8,0.562,38.3,11.9,8.0,8.6,8.8,1098.0,2279.0,0.482,0.484,3041.0,Yes
8869,8869,1988.0,Michael Jordan*,SG,24.0,3311.0,31.7,0.603,34.1,15.2,9.8,12.2,11.8,1069.0,1998.0,0.535,0.537,2868.0,No
9297,9297,1989.0,Michael Jordan*,SG,25.0,3255.0,31.1,0.614,32.1,14.6,9.8,12.6,12.0,966.0,1795.0,0.538,0.546,2633.0,No
9747,9747,1990.0,Michael Jordan*,SG,26.0,3197.0,31.2,0.606,33.7,14.7,9.7,10.6,10.1,1034.0,1964.0,0.526,0.550,2753.0,Yes
10212,10212,1991.0,Michael Jordan*,SG,27.0,3034.0,31.6,0.605,32.9,14.9,8.9,10.8,9.8,990.0,1837.0,0.539,0.547,2580.0,No
10666,10666,1992.0,Michael Jordan*,SG,28.0,3102.0,27.7,0.579,31.7,12.1,6.9,8.6,8.3,943.0,1818.0,0.519,0.526,2404.0,No
11106,11106,1993.0,Michael Jordan*,SG,29.0,3067.0,29.7,0.564,34.7,12.0,8.3,9.5,8.9,992.0,2003.0,0.495,0.515,2541.0,Yes
12048,12048,1995.0,Michael Jordan*,SG,31.0,668.0,22.1,0.493,33.2,1.2,2.0,2.7,0.8,166.0,404.0,0.411,0.431,457.0,Yes
